# 聊天机器人
 
在本笔记本中，您将探索如何利用聊天格式与专门针对特定任务或行为定制的聊天机器人进行扩展对话。

[website](https://learn.deeplearning.ai/chatgpt-prompt-eng/lesson/8/chatbot)

## Setup

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [9]:
messages =  [  
{'role':'system', 'content':'你是一个说话像文学家鲁迅的智能助手。'},    
{'role':'user', 'content':'讲一个笑话'},   
{'role':'assistant', 'content':'怎么才能把一片面包变成一个蜜蜂？把它扔到蜂窝里！'},   
{'role':'user', 'content':'为什么'},  
]

In [10]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

因为蜜蜂会围着那片面包飞来飞去啊！这是一个比较幽默的笑话，希望你喜欢。


In [11]:
messages =  [  
{'role':'system', 'content':'你是一个友好的聊天机器人.'},    
{'role':'user', 'content':'你好，我的名字是安庆'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

你好，安庆！很高兴见到你。有什么我可以帮助你的吗？


In [14]:
messages =  [  
{'role':'system', 'content':'你是一个友好的聊天机器人.'},    
{'role':'user', 'content':'好的，你可以记住我的名字。你的名字是什么？'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

我的名字是AI助手，很高兴认识你！请问有什么我可以帮你的吗？


In [15]:
messages =  [  
{'role':'system', 'content':'你是一个友好的聊天机器人.'},
{'role':'user', 'content':'你好，我的名字是安庆。'},
{'role':'assistant', 'content': "你好，安庆！很高兴见到你。有什么我可以帮助你的吗？"},
{'role':'user', 'content':'是的，你可以记住我，我的名字是？'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

当然可以，你的名字是安庆。我会记住的。


## 订单机器人

我们可以自动收集用户提示和助手响应，以构建一个订购机器人（OrderBot）。OrderBot 将在披萨餐厅接受订单。

In [16]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
您是 OrderBot，这是一项为比萨餐厅收集订单的自动化服务。 \
你先问候客户，然后收集订单，\
然后问是自取还是送货。 \
您等待收集整个订单，然后对其进行汇总并检查最终\
如果客户想要添加任何其他内容，则需要时间。 \
如果是送货，你要一个地址。 \
最后你收款了。\
确保清楚地说明所有选项、附加功能和尺寸，以独特地\
从菜单中识别项目。\
你以简短、非常友好的对话方式回应。 \
菜单包括\
意大利辣香肠披萨 12.95, 10.00, 7.00 \
芝士披萨 10.95, 9.25, 6.50 \
茄子披萨 11.95, 9.75, 6.75 \
薯条 4.50, 3.50 \
希腊沙拉 7.25 \
配料：\
额外的奶酪 2.00, \
蘑菇 1.50 \
香肠 3.00 \
加拿大培根 3.50 \
艾酱1.50\
辣椒 1.00 \
饮料：\
可乐 3.00, 2.00, 1.00 \
精灵 3.00, 2.00, 1.00 \
瓶装水 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':"""
创建一个关于之前食品订单的JSON摘要。 逐项列出每一项的价格字段应该是：
1) 一个比萨饼，包括配菜 
2) 配料表
3) 饮料表
4) 配菜清单
5) 总价格
"""
 },    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)